Imports

In [ ]:
from utils import *
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data

import numpy as np
import torch

import matplotlib.pyplot as plt

In [ ]:
from scipy.sparse import load_npz
import csv
import os

Initialize Autoencoders

In [ ]:
class AutoEncoder(nn.Module):
    torch.manual_seed(42)
    def __init__(self, num_question, k=100):
        """ Initialize a class AutoEncoder.

        :param num_question: int
        :param k: int
        """
        super(AutoEncoder, self).__init__()

        # Define linear functions.
        self.encoder = nn.Linear(num_question, k) #self.g
        self.decoder = nn.Linear(k, num_question) #self.h

    def get_weight_norm(self):
        """ Return ||W^1||^2 + ||W^2||^2.

        :return: float
        """
        g_w_norm = torch.norm(self.encoder.weight, 2) ** 2
        h_w_norm = torch.norm(self.decoder.weight, 2) ** 2

        return g_w_norm + h_w_norm

    def forward(self, inputs):
        """ Return a forward pass given inputs.

        :param inputs: user vector.
        :return: user vector.
        """

        x = torch.sigmoid(self.encoder(inputs))
        out = torch.sigmoid(self.decoder(x))

        return out

Training Function

In [ ]:
def train_bagging(model, lr, lamb, train_data, zero_train_data, valid_data, num_epoch):
    """ Train the neural network, where the objective also includes
    a regularizer.

    :param model: Module
    :param lr: float
    :param lamb: float
    :param train_data: 2D FloatTensor
    :param zero_train_data: 2D FloatTensor
    :param valid_data: Dict
    :param num_epoch: int
    :return: None
    """

    # TODO: Add a regularizer to the cost function.
    norm = model.get_weight_norm()
    norm = norm.detach().numpy()
    print("norm = ", norm)

    # Tell PyTorch you are training the model.
    model.train()

    # Define optimizers and loss function.
    optimizer = optim.SGD(model.parameters(), lr=lr)
    num_student = train_data.shape[0]

    for epoch in range(0, num_epoch):
        train_loss = 0.

        for user_id in range(num_student):
            inputs = Variable(zero_train_data[user_id]).unsqueeze(0)  #answers to all questions by a student
            target = inputs.clone()

            optimizer.zero_grad()
            output = model(inputs)

            # Mask the target to only compute the gradient of valid entries.
            nan_mask = np.isnan(train_data[user_id].unsqueeze(0).numpy())
            target[0][nan_mask] = output[0][nan_mask]

            loss = torch.sum(((output - target) ** 2.)) + lamb*norm/2
            loss.backward()

            train_loss += loss.item()
            optimizer.step()

        valid_acc = evaluate(model, zero_train_data, valid_data)
        print("Epoch: {} \tTraining Cost: {:.6f}\t "
              "Valid Acc: {}".format(epoch, train_loss, valid_acc))


Accuracy Evaluation Function

In [ ]:
def evaluate_bagging(model1, model2, model3, train_data, valid_data):
    """ Evaluate the valid_data on the current model.

    :param model: Module
    :param train_data: 2D FloatTensor
    :param valid_data: A dictionary {user_id: list,
    question_id: list, is_correct: list}
    :return: float
    """
    # Tell PyTorch you are evaluating the model.
    model.eval()

    total = 0
    correct = 0

    for i, u in enumerate(valid_data["user_id"]):
        inputs = Variable(train_data[u]).unsqueeze(0)
        output1 = model1(inputs)
        output2 = model2(inputs)
        output3 = model3(inputs)

        guess = (((output1[0][valid_data["question_id"][i]].item() >= 0.5)+(output2[0][valid_data["question_id"][i]].item() >= 0.5)+(output3[0][valid_data["question_id"][i]].item() >= 0.5))/3) >= 0.5

        if guess == valid_data["is_correct"][i]:
            correct += 1
        total += 1
    return correct / float(total)

Bootstrapping Training Data

In [ ]:
#create list of randomly sorted iterables (equal to # of students)
import random

def bootstrap(train_matrix):
  '''Input: np train_matrix
  Returns: tensor zero_train_matrix2, train_matrix2, zero_train_matrix3,train_matrix3
  '''

  train_matrix2 = train_matrix.copy()
  train_matrix3 = train_matrix.copy()

  rand_list1 = list(range(train_matrix.shape[0]))
  random.shuffle(rand_list1)
  rand_list2 = list(range(train_matrix.shape[0]))
  random.shuffle(rand_list2)

  for i in range(train_matrix.shape[0]):
    j = rand_list1[i]
    k = rand_list2[i]

    train_matrix2[i] = train_matrix[j]
    train_matrix3[i] = train_matrix[k]

  zero_train_matrix2 = train_matrix2.copy()
  zero_train_matrix3 = train_matrix3.copy()

  #Fill in the missing entries to 0.
  zero_train_matrix2[np.isnan(train_matrix2)] = 0
  zero_train_matrix3[np.isnan(train_matrix3)] = 0

  # Change to Float Tensor for PyTorch.
  zero_train_matrix2 = torch.FloatTensor(zero_train_matrix2)
  train_matrix2 = torch.FloatTensor(train_matrix2)

  zero_train_matrix3 = torch.FloatTensor(zero_train_matrix3)
  train_matrix3 = torch.FloatTensor(train_matrix3)

  return zero_train_matrix2, train_matrix2, zero_train_matrix3, train_matrix3

Main Functions

In [ ]:
if __name__ == "__main__":


  zero_train_matrix1, train_matrix1, train_data, valid_data, test_data, np_train_matrix = load_data_for_bagging()
  zero_train_matrix2, train_matrix2, zero_train_matrix3, train_matrix3 = bootstrap(np_train_matrix)


  # Set model hyperparameters.
  k = 50
  num_questions = zero_train_matrix1.shape[1]

  # Set optimization hyperparameters.
  lr1 = 0.01; lr2 = 0.02; lr3 = 0.001
  num_epoch1 = 40; num_epoch2 = 30; num_epoch3 = 40
  lamb1 = 0.001; lamb2 = 0.001; lamb3 = 0.001

  #initialize models
  modelb1 = AutoEncoder(num_question = num_questions, k = k); modelb2 = AutoEncoder(num_question = num_questions, k = k); modelb3 = AutoEncoder(num_question = num_questions, k = k)

  #train the models on the three training datasets
  train_bagging(modelb1, lr1, lamb1, train_matrix1, zero_train_matrix1, valid_data, num_epoch1)
  train_bagging(modelb2, lr2, lamb2, train_matrix2, zero_train_matrix2, valid_data, num_epoch2)
  train_bagging(modelb3, lr3, lamb3, train_matrix3, zero_train_matrix3, valid_data, num_epoch3)

  #make ensembled validation predictions and evaluate accuracy (only one epoch -- because we have the fully trained model)
  val_acc_b = evaluate_bagging(modelb1, modelb2, modelb3, zero_train_matrix1,valid_data) #choice of train matrix doesn't matter because eval function takes the ztm list of user ids and makes predictions on that specific list

  #make ensembled test predictions and evaluate accuracy (only one epoch -- because we have the fully trained model)
  test_acc_b = evaluate_bagging(modelb1, modelb2, modelb3, zero_train_matrix1,test_data)

  print("Validation Accuracy = ", val_acc_b, "     ", "Test Accuracy = ", test_acc_b)